In [ ]:
print(91*'-')
print('INICIO DEL PROGRAMA')
qqqq = [8,9,16,25,27] #Lista de potencias no triviales de números primos
for q in qqqq:
#################################################################################
#### k, a, ma, elem, menoselem
    print(91*'-')
    print('INICIO DEL CASO Fq con q =',q)
    k.<a> = GF(q)
    ma = a.charpoly('y')
    print(31*'-')
    print('El polinomio caracteristico es',ma)
    elem = []
    if is_prime(q) == True :
        for i in k:
            if i == 0 :
                elem.append(i)
            else :
                elem.append(i)
    else :
        for i in range(q):
            if i == 0 :
                elem.append(a*0)
            else :
                elem.append(a^i)
    menoselem = []
    for eee in elem:
        menoselem.append(-eee)
#################################################################################
#### R, x, monicirred2
    R.<x> = PolynomialRing(k)
    monicirred2 = []
    for pol in R.polynomials(2):
        if pol.is_irreducible():
            if list(pol)[2] == 1:
                monicirred2.append(pol)
    print(31*'-')
    print('Hay',len(monicirred2),'polinomios monicos irreducibles de grado 2')
#################################################################################
#### K, O
    K.<x> = FunctionField(k)
    O = K.maximal_order()
#################################################################################
#### MS, IMS
    MS = MatrixSpace(k,2)
    IMS = [] #Matrices que no fijan lugares racionales
    for pmi in monicirred2:
        pmilista = list(pmi)
        a0 = pmilista[0]
        a1 = pmilista[1]
        for t in k:
            A = [t,-a0,1,a1+t]
            B = Matrix(k,2,A)
            if B.det() != 0 :
                IMS.append(B)
#################################################################################
#### matord
    matord = []
    for matriz in IMS:
        n = 1
        brk = 0
        while brk == 0 :
            if matriz^n == (matriz^n)[0][0]*identity_matrix(k, 2):
                matord.append(n)
                brk = 1
            else:
                n = n+1
#################################################################################
#### NN, N
    NN = Set(matord)
    N = []
    for n in NN:
        if ( n < q + 1 ) and ( n > 3 ) and (n.divides(q+1)) :
            N.append(n)
    print(31*'-')
    print('Las longitudes posibles para codigos sigma ciclicos son',N)
#################################################################################
    MGCNEparacadan = [] 
    for n in N:
        print(31*'-')
        print('ESTUDIAREMOS AHORA CODIGOS DE LONGITUD',n,'Y DIVISORES G DE GRADO 2')
        cantmatn = 0 
        matn = [] 
        for i in range(len(matord)):
            if matord[i] == n :
                cantmatn = cantmatn + 1
                matn.append(IMS[i])
        print(31*'-')
        print('Hay',cantmatn,'matrices de orden' ,n, 'sin reducir PGL2')
#################################################################################
        MATGEN = []
        cantlugfijos = []
        CONTEO = []
#################################################################################
        for B in matn:
            Binv = Matrix(k,2,[B[1][1],-B[0][1],-B[1][0],B[0][0]])
            sigma = K.hom((B[0][0]*x+B[0][1])/(B[1][0]*x+B[1][1]))
            #BUSCAMOS LOS LUGARES DE GRADO 2 FIJOS POR SIGMA
            lg2fijos = []
            for pol in monicirred2:
                sigmapol = sigma(pol)
                arriba = sigmapol.numerator()
                abajo = sigmapol.denominator()
                testeo = [((pol).numerator()).divides(arriba), ((pol).numerator()).divides(abajo)]
                if testeo == [True, False] :
                    lg2fijos.append(pol)
            cantlugfijos.append(len(lg2fijos))
            #print('-------------------------------')
            #print('Existen',len(lg2fijos),'lugares de grado 2 fijos por sigma asociado a la matriz')
            #print(B)
            #print('lg2fijos=',lg2fijos)
            #BUSCAMOS LAS ORBITAS DE LONGITUD n ASOCIADAS A SIGMA
#################################################################################
            orbitas = []
            for i in range(q):
                for hache in range(q):
                    if menoselem[i] == elem[hache]:
                        j = hache
                brk = 0
                for orb in orbitas:
                    if K.places_finite()[j] in orb:
                        brk = brk + 1
                if brk == 0:
                    orbita = [K.places_finite()[j]]
                    aux = [a^i]
                    aux2 = [a^j]
                    brk2 = 0
                    while brk2 == 0 and len(orbita)<n :
                        num = (Binv[0][0]*aux[len(aux)-1] + Binv[0][1])
                        deno = (Binv[1][0]*aux[len(aux)-1] + Binv[1][1])
                        if deno != 0:
                            new = num/deno
                            aux.append(new)
                            indice = 0
                            while new != menoselem[indice]:
                                indice = indice + 1
                            orbita.append(K.places_finite()[indice])
                        else:
                            orbita.append(K.places()[0])
                            brk2 = 1
                    if len(orbita) == n:
                        #print(orbita)
                        orbitas.append(orbita)
#################################################################################
            #CONSTRUIR LOS CODIGOS C_l(ORBITA,g)
            for Q in lg2fijos:
                #print(Q)
                #print(Q in R , Q in monicirred2 , Q in K)
                I = O.ideal(Q)
                G = I.divisor_of_zeros()
                #print(G)
                Base = (G).basis_function_space()
                #print('La base del espacio de Riemann-Roch asociado a G=',Q,'es')
                #print(Base)
                Matgen = []
                for orbita in orbitas:
                    lista = []
                    for base in Base:
                        for orb in orbita:
                            lista.append((base).evaluate(orb))
                    #print(lista)
                    matriz = Matrix(k,len(Base),len(orbita),lista)
                    #print(matriz)
                    Matgen.append(matriz)
                Cod = []
                for matgen in Matgen:
                    cod = LinearCode(matgen)
                    Cod.append(cod)
                #print(Cod)
                MAT = []
                for cod in Cod:
                    MAT.append(cod.systematic_generator_matrix())
                    MATGEN.append(cod.systematic_generator_matrix())
                    #print(cod.systematic_generator_matrix())
                conteo = len(list(Set(MAT)))
                CONTEO.append(conteo)
        Theta = list(Set(MATGEN))
        theta = len(Theta)
        print(31*'-')
        print('HAY', theta, 'MATRICES GENERADORAS DE CODIGOS DISTINTOS, PERO QUIZAS EQUIVALENTES:')
        if theta != 0:
            noequiv = 1
            for mu in Theta:
                for nu in Theta:
                    C1 = codes.LinearCode(mu)
                    C2 = codes.LinearCode(nu)
                    if not C1.is_permutation_equivalent(C2):
                        noequiv = noequiv + 1
            if noequiv == 1 :
                print(31*'-')
                print('Existe un UNICO codigo sigma-ciclico, salvo equivalencias, de longitud', n)
                print('y dimension 3 sobre Fq con q =',q)
                MGCNEparacadan.append([Theta[0]])
            else:
                mgcne=[]
                mgcnep=[]
                codigos = []
                for mu in Theta:
                    C1 = codes.LinearCode(mu)
                    brk = 0
                    for code in range(len(codigos)):
                        C2 = codigos[code]
                        if C1.is_permutation_equivalent(C2):
                            brk = brk + 1
                    if brk == 0:
                        codigos.append(C1)
                        mgcnep.append(mu)
                print(31*'-')
                print('Existen',len(codigos),'No equivalentes por permutaciones')
                print('Las matrices generadoras de estos codigos son:')
                for i in range(len(mgcnep)):
                    print(mgcnep[i])
                print('Debemos chequear si son o no monomialmente equivalentes')
#################################################################################
                recuento = 0
                CCC2 = codes.LinearCode(mgcnep[1])
                for fila in MatrixSpace(k,1,n):
                    control = 1
                    for pos in range(n):
                        control = control*fila.row(0).list()[pos]
                    if control != 0 :
                        Diago = diagonal_matrix(k,fila.row(0).list())
                        product = mgcnep[0]*Diago
                        CCC1 = codes.LinearCode(product)
                        if CCC1.is_permutation_equivalent(CCC2):
                            recuento = recuento + 1
#################################################################################
                print(31*'-')
                print('Existen',len(codigos)-recuento,' codigos sigma-ciclicos, salvo equivalencias,')
                print(' de longitud', n,'y dimension 3 sobre Fq con q =',q)
                MGCNEparacadan.append(mgcnep)
        print(31*'-')
        print('Las matrices generadoras de codigos no equivalentes son')
        for j in range(len(MGCNEparacadan[-1])):
            print(31*'-')
            print((MGCNEparacadan[-1])[j])
    print(31*'-')
    print('FIN DEL CASO Fq con q =',q)
print(91*'-')
print('FIN DEL PROGRAMA')
print(91*'-')

-------------------------------------------------------------------------------------------
INICIO DEL PROGRAMA
-------------------------------------------------------------------------------------------
INICIO DEL CASO Fq con q = 8
-------------------------------
El polinomio caracteristico es y^3 + y + 1
-------------------------------
Hay 28 polinomios monicos irreducibles de grado 2
-------------------------------
Las longitudes posibles para codigos sigma ciclicos son []
-------------------------------
FIN DEL CASO Fq con q = 8
-------------------------------------------------------------------------------------------
INICIO DEL CASO Fq con q = 9
-------------------------------
El polinomio caracteristico es y^2 + 2*y + 2
-------------------------------
Hay 36 polinomios monicos irreducibles de grado 2
-------------------------------
Las longitudes posibles para codigos sigma ciclicos son [5]
-------------------------------
ESTUDIAREMOS AHORA CODIGOS DE LONGITUD 5 Y DIVISORES G DE

In [8]:
elem

[0, 1, 2, 3, 4, 5, 6]

In [9]:
menoselem

[0, 6, 5, 4, 3, 2, 1]

In [4]:
is_prime(7)

True

In [5]:
is_prime(9)

False